# Graph feature extraction

## Setup

Install libraries:
- standard data science stack (Numpy, Pandas, Scikit-learn, tqdm)
- graph libraries (NetworkX, NetworKit)
- deep learning on graphs (PyTorch, PyTorch Scatter, PyTorch Geometric)

For PyTorch Scatter, we use a small workaround for Google Colab - change PyTorch or Python version if necessary.

In [ ]:
!pip install numpy pandas scikit-learn tqdm networkx networkit torch torch_geometric --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0%2Bcpu/torch_scatter-2.1.2%2Bpt21cpu-cp310-cp310-linux_x86_64.whl

## Data loading

We will use **IMDB-BINARY** dataset:
- a movie collaboration dataset
- each graph represents a set of actors playing in movies from genre: Action or Romance
- graphs are **ego-networks** (who-knows-whom), nodes are actors, edges mean that actors appear in the same movie
- statistics:
  - 1000 graphs
  - 2 classes
  - avg # nodes: 19.8
  - avg # edges: 193.1
  - no node/edge features

So we have to predict the genre of the movies, based only on actors interactions!

Dataset hosted by TU Dortmund University at [TUDataset](https://chrsmrrs.github.io/datasets/docs/datasets/).

In [ ]:
import pandas as pd
from torch_geometric.data import Dataset
from torch_geometric.datasets import TUDataset


dataset = TUDataset(
    root="data",  # where to save dataset
    name="IMDB-BINARY"
)

print(f"Number of classes: {dataset.num_classes}")
print()
dataset.print_summary()
print()
pd.Series(dataset.y).value_counts().plot.bar(title="Class distribution")

### PyTorch Geometric Dataset

1. Built from graphs
2. Regular PyTorch dataset, with indexing etc.
3. Collection of `Data` objects, representing graphs:
   - `edge_index` - adjacency matrix in COO sparse format
   - `num_nodes` - $|V|$
   - `x` - node features
   - `y` - class

In [ ]:
graph = dataset[0]

print("Graph:", graph)
print("Adjacency matrix (COO sparse format):", graph.edge_index)
print("Number of nodes:", graph.num_nodes)
print("Node features:", graph.x)
print("Class:", graph.y)

## Local Degree Profile (LDP)

1. Extract LDP features
2. Gather features for all graphs
3. Classify with Random Forest

We will use [PyTorch Geometric code for LDP](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/transforms/local_degree_profile.html), but slightly modified.

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn.aggr.fused import FusedAggregation
from torch_geometric.utils import degree


def extract_ldp_features(graph: Data) -> np.ndarray:
    row, col = graph.edge_index
    N = graph.num_nodes

    # compute degree for each node, using adjacency matrix

    # compute degree statistics, using node degrees and adjacency matrix

    # combine features into a single list of Numpy vectors

    # aggregate each feature with a histogram and concatenate them

    return aggregated_features


In [ ]:
from tqdm.notebook import tqdm


def dataset_to_ldp_features(dataset: Dataset) -> np.ndarray:
    X = []
    for graph in tqdm(dataset, total=len(dataset)):
        x = extract_ldp_features(graph)
        X.append(x)

    return np.stack(X)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


def train_ldp(dataset: Dataset) -> None:
    clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
    X = dataset_to_ldp_features(dataset)
    y = dataset.y.numpy()

    scores = cross_val_score(
        estimator=clf,
        X=X,
        y=y,
        scoring="accuracy",
        cv=10,
    )
    test_acc = np.mean(scores)
    print(f"Test accuracy: {100 * test_acc:.2f}")


In [ ]:
train_ldp(dataset)

## Local Topological Profile (LTP)

1. Extract LDP features
2. Transform to NetworKit graph
3. Calculate additional LTP features
4. Gather features for all graphs
5. Classify with Random Forest

We will use NetworKit tutorials:
- [edge betweenness centrality](https://networkit.github.io/dev-docs/notebooks/Centrality.html)
- [Jaccard index](https://networkit.github.io/dev-docs/notebooks/LinkPrediction.html)
- [Local Degree Score](https://networkit.github.io/dev-docs/notebooks/Sparsification.html)

In [ ]:
from networkit.centrality import Betweenness
from networkit.graph import Graph as NetworkitGraph
from networkit.linkprediction import JaccardIndex
from networkit.nxadapter import nx2nk
from networkit.sparsification import LocalDegreeScore
from torch_geometric.utils import to_networkx


def calculate_edge_betweenness(graph: NetworkitGraph) -> np.ndarray:
    # calculate betweenness and get edge scores

    # to Numpy
    scores = np.array(scores, dtype=np.float32)
    return scores


def calculate_jaccard_index(graph: NetworkitGraph) -> np.ndarray:
    # calculate Jaccard index values

    # to Numpy, remove infinite values
    scores = np.array(scores, dtype=np.float32)
    scores = scores[np.isfinite(scores)]
    return scores


def calculate_local_degree_score(graph: NetworkitGraph) -> np.ndarray:
    # calculate Local Degree Score values

    # to Numpy
    scores = np.array(scores, dtype=np.float32)
    return scores


def extract_ltp_features(graph: Data) -> np.ndarray:
    row, col = graph.edge_index
    N = graph.num_nodes

    # compute degree for each node, using adjacency matrix

    # compute degree statistics, using node degrees and adjacency matrix

    # combine features into a single list of Numpy vectors

    # transform to NetworKit
    graph_networkx = to_networkx(graph, to_undirected=True)
    graph_networkit = nx2nk(graph_networkx)
    graph_networkit.indexEdges()

    # calculate additional features
    ebc = calculate_edge_betweenness(graph_networkit)
    jaccard_index = calculate_jaccard_index(graph_networkit)
    lds = calculate_local_degree_score(graph_networkit)

    features.extend([ebc, jaccard_index, lds])

    # aggregate each feature with a histogram and concatenate them


    return aggregated_features


In [ ]:
def dataset_to_ltp_features(dataset: Dataset) -> np.ndarray:
    X = []
    for graph in tqdm(dataset, total=len(dataset)):
        x = extract_ltp_features(graph)
        X.append(x)

    return np.stack(X)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


def train_ltp(dataset: Dataset) -> None:
    clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
    X = dataset_to_ltp_features(dataset)
    y = dataset.y.numpy()

    scores = cross_val_score(
        estimator=clf,
        X=X,
        y=y,
        scoring="accuracy",
        cv=10,
    )
    test_acc = np.mean(scores)
    print(f"Test accuracy: {100 * test_acc:.2f}")


In [ ]:
train_ltp(dataset)